# Word2vec: работаем с векторными моделями в Python

*Эта тетрадка — переосмысленная, дополненная и местами упрощенная мной версия туториала по вордтувеку от Лизы Кузьменко, которая со-основала RusVectores вместе с Андреем Кутузовым*


**Word2vec** - библиотека для получения векторных представлений слов на основе их совместной встречаемости в текстах. Вы можете освежить в памяти механизмы работы **word2vec**, прочитав [эту статью](https://vk.com/@sysblok-word2vec-pokazhi-mne-svoi-kontekst-i-ya-skazhu-kto-ty). 

Сейчас мы научимся использовать **word2vec** в своей повседневной работе. Мы будем использовать реализацию **word2vec** в библиотеке [gensim](https://radimrehurek.com/gensim/) для языка программирования **python**.

Для работы с эмбеддингами слов существуют и другие библиотеки: кроме [gensim](https://radimrehurek.com/gensim/) можно делать векторные модели в [keras](https://keras.io/), [tensorflow](https://www.tensorflow.org/), [pytorch](https://pytorch.org/). Но мы будем работать с *gensim*, потому что тут это проще и потому что создатели библиотеки моделей RusVectores затачивались под нее.


***Gensim***  - изначально библиотека для тематического моделирования текстов. Однако помимо различных алгоритмов для *topic modeling* в ней реализованы на **python** и алгоритмы из тулкита **word2vec** (который в оригинале был написан на C++). Если вы работаете на своей машине и **gensim** у вас не установлен, нужно его установить: `pip3 install gensim`

В колабе генсим установлен по умолчанию


In [ ]:
import gensim

Тьюториал состоит из двух частей:
* В первой части мы разберёмся, как загружать уже готовые векторные модели и работать с ними. Например, мы научимся выполнять простые операции над векторами слов, такие как «найти слово с наиболее близким вектором» или «вычислить коэффициент близости между двумя векторами слов». Также мы рассмотрим более сложные операции над векторами, например, «вычесть из вектора слова вектор другого слова», «прибавить к вектору слова вектор другого слова»  «найти лишний вектор в группе слов».

* Во второй части мы научимся предобрабатывать текстовые файлы и самостоятельно тренировать векторную модель на своих данных.

## 1. Работа с готовыми векторными моделями при помощи библиотеки Gensim

Для своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами (об этом раздел 2). Но для каких-то общих целей уже есть готовые модели, в т.ч. для русского языка, обученные на больших корпусах

Модели для русского скачать можно здесь - https://rusvectores.org/ru/models/

Существуют несколько форматов, в которых могут храниться модели. Во-первых, данные могут храниться в нативном формате *word2vec*, при этом модель может быть бинарной или не бинарной. Для загрузки модели в формате *word2vec* в классе `KeyedVectors` (в котором хранится большинство относящихся к дистрибутивным моделям функций) существует функция `load_word2vec_format`, а бинарность модели можно указать в аргументе `binary` (внизу будет пример). Помимо этого, модель можно хранить и в собственном формате *gensim*, для этого существует класс `Word2Vec` с функцией `load`. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте. Наш код определяет тип модели по её расширению, но вообще файл с моделью может называться как угодно, жестких ограничений для расширения нет.

Давайте скачаем новейшую модель для русского языка, созданную на основе [Национального корпуса русского языка (НКРЯ)](http://www.ruscorpora.ru/) (поскольку zip-архив с моделью весит почти 500 мегабайт, следующая ячейка выполнится у вас не сразу!). 


In [ ]:
!wget 'http://vectors.nlpl.eu/repository/20/180.zip'

Теперь моделька в виде zip-архива лежит у нас в рабочей папке

In [ ]:
!ls

Распаковывать скачанный архив для обычных моделей не нужно, так как его содержимое прочитается при помощи специальной инструкции:

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('180.zip', 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

### Все, этой моделью уже можно пользоваться для оценки семантической близости:

Выводим 10 соседей слова по близости и меру близости с ними — метод `most_similar`:

In [ ]:
model.most_similar ('кофе_NOUN')

⛳ 💻  Вопрос: как получить больше 10?

In [ ]:
## ваш код
model.most_similar ('лингвист_NOUN', topn=25)

In [ ]:
model.most_similar ('программирование_NOUN')

In [ ]:
model.most_similar ('программировать_VERB', topn=16)

### 🤔 стоп, а что за _NOUN, _VERB это мне всегда руками так писать?

Cлова в модель надо подавать с указанием части речи (pos tag) из набора тегов [Universal POS-tags](https://universaldependencies.org/u/pos/). Так устроены модели RusVectores — в них снята омонимия на уровне словоформ:

In [ ]:
model.most_similar ('печь_NOUN')

In [ ]:
model.most_similar ('печь_VERB')

Давайте пока работаем с небольшим числом слов писать это руками. А дальше поговорим, как с этим работать автоматически, если анализируем большой текст. 

Если мы прогоняем много слов, стоит вставить проверку, что слова нет в модели. Допустим, нам интересны такие слова (пример для русского языка):

In [ ]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студент_ADJ']

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [ ]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

Наш код сказал нам, что прилагательного студент не модель не знает...

### сравнить близость 2 слов:

Находим косинусную близость пары векторов слов — метод `similarity`:

In [ ]:
print(model.similarity('кофе_NOUN', 'чай_NOUN'))

In [ ]:
print(model.similarity('компот_NOUN', 'чай_NOUN'))

In [ ]:
print(model.similarity('чай_NOUN', 'картофель_NOUN'))

<img src="https://www.meme-arsenal.com/memes/32687f97c291d55dc3143e26821ff4d4.jpg">

Мистер картошка, вы раскрыты! 

## 2. Более сложные операции над векторами

Помимо более простых операций над векторами (нахождение косинусной близости между двумя векторами и ближайших соседей вектора) **gensim** позволяет выполнять и более сложные операции над несколькими векторами. Так, например, мы можем найти лишнее слово в группе. Лишним словом является то, вектор которого наиболее удален от других векторов слов.

In [ ]:
words = ['яблоко_NOUN',
 'груша_NOUN',
 'виноград_NOUN',
 'банан_NOUN',
 'лимон_NOUN',
 'картофель_NOUN', 'лошадь_NOUN', 'философия_NOUN', 'видеоблог_NOUN']

In [ ]:
print(model.doesnt_match(words))

Также можно складывать и вычитать вектора нескольких слов. Например, сложив два вектора и вычтя из них третий вектор, мы можем решить своеобразную пропорцию. Подробнее о семантических пропорциях вы можете прочитать в [материале Системного Блока](https://vk.com/@sysblok-vo-chto-prevraschaetsya-zhizn-bez-lubvi).

Меня всегда радует, что вот это реально работает: 
🦅 -> 🐠

In [ ]:
print(model.most_similar(positive=['машина_NOUN', 'крыло_NOUN'], negative=['колесо_NOUN'])[0][0])

В обратную сторону тоже: 🐠 -> 🦅 

In [ ]:
print(model.most_similar(positive=['птица_NOUN', 'плавник_NOUN'], negative=['крыло_NOUN'])[0][0])

Ну и конечно: 💔

In [ ]:
print(model.most_similar(positive=['жизнь_NOUN'], negative=['любовь_NOUN'])[0][0])

### ⛳ 💻  Задание: подберите еще 3-4 симпатичных примера векторной арифметики

## 3. Предобработка текстовых данных

Вернемся к вопросу о pos-тегах (_NOUN, _VERB и проч). Пока мы обходились без них, но это выглядело как костыль, правда же 🔩 Если мы хотим обрабатывать свои тексты — нам надо бы научиться предобрабатывать их так, чтобы каждое слово шло именно с таким тегом. Тогда можно будет гонять на них модели word2vec от RusVectores (а это лучшее что есть для русского и вообще такой стандартный стандарт).  

Предобработка текстов для тренировки моделей выглядит следующим образом:
* сначала мы приведем все слова к начальной форме (лемматизируем) и удалим стоп-слова;
* затем мы приведем все леммы к нижнему регистру;
* для каждого слова добавим его частеречный тэг.

Давайте попробуем воссоздать процесс предобработки текста на примере [сказки Хармса](https://raw.githubusercontent.com/dhhse/dh2020/master/data/harms.txt). Для предобработки можно использовать различные тэггеры, мы сейчас будем использовать [*UDPipe*](https://ufal.mff.cuni.cz/udpipe), чтобы сразу получить частеречную разметку в виде Universal POS-tags. Сначала установим обертку *UDPipe* для Python:

In [ ]:
!pip3 install ufal.udpipe

*UDPipe* использует предобученные модели для лемматизации и тэггинга. Вы можете использовать [уже готовую модель](https://rusvectores.org/static/models/udpipe_syntagrus.model) или обучить свою. 

Кусок кода ниже скачает модель UDPipe для лингвистической предобработки. Модель весит 40 мегабайт, поэтому ячейка может выполнятся некоторое время, особенно если у вас небыстрый интернет. 

In [ ]:
!wget 'https://rusvectores.org/static/models/udpipe_syntagrus.model'

In [ ]:
! ls

In [ ]:
modelfile = 'udpipe_syntagrus.model'

## ⚠️ Для соответствия моделям RusVectores требуется еще немножко допиливания напильником поверх UDPipe. Да и сама машинерия UDPipe довольно громоздко устроена (там выдача [в формате CONLLU](https://universaldependencies.org/format.html)). Поэтому ниже я просто переиспользую функции, которые написала со-авторка RusVectores Лиза Кузьменко для предобработки при помощи UDPipe и использую их. Но в целом там вроде бы ничего магического не происходит:

Приступим к собственно предобработке текста. Попробуем лемматизировать текст и добавить частеречные тэги при помощи этой функции:

In [ ]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn


Эту функцию можно также изменить под конкретную задачу. Например, если частеречные тэги нам не нужны, в функции ниже выставим `keep_pos=False`. Если необходимо сохранить знаки пунктуации, можно выставить `keep_punct=True`. 

Теперь загружаем модель *UDPipe*, читаем текстовый файл и обрабатываем его при помощи нашей функции. В файле должен содержаться необработанный текст (одно предложение на строку или один абзац на строку).
Этот текст токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход мы получаем последовательность разделенных пробелами лемм с частями речи ("зеленый\_NOUN трамвай\_NOUN").

In [ ]:
# в функции ниже используется питоновский модуль wget (не то же самое, что !wget выше)
# на тот случай, если модель не скачана -- он ее автоматически перескачает
# поэтому в этой ячейке установим питоновский wget
!pip3 install wget

In [ ]:
from ufal.udpipe import Model, Pipeline
import os
import re
import sys
import wget

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    lines = text.split('\n')
    tagged = []
    for line in lines:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        tagged_line = ' '.join(output)
        tagged.append(tagged_line)
    return '\n'.join(tagged)

def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken

Скачаем теперь текст, с которым будем работать:

In [ ]:
!wget 'https://raw.githubusercontent.com/dhhse/dh2020/master/data/harms.txt'

In [ ]:
text = open('harms.txt', 'r', encoding='utf-8').read()

processed_text = tag_ud(text=text, modelfile=modelfile)

print(processed_text[:350])

with open('harms_processed.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)

Наша функция напечатает обработанный текст, который мы теперь можем также сохранить в файл. 

Итак, в ходе этой части тьюториала мы научились от "сырого текста" приходить к лемматизированному тексту с частеречными тэгами, который уже можно подавать на вход модели! 




In [ ]:
with open('harms_processed.txt', 'r', encoding='utf-8') as tagged_text:
    words = tagged_text.read().split()
print(len(words))




In [ ]:
 for word in words[140:150]:
      print ('слово: ', word)
      if word in model:
        x = model.most_similar(word)
        print ('ближайший синоним: ', x[0][0],"\n")

[Вернемся к слайдам ненадолго](https://docs.google.com/presentation/d/11fYkNG1IFBJzVQ27LNxaE_fj8XHO40JxYaUUSzpcCKQ/edit#slide=id.gd5da728dca_0_1009) — нам осталась fun part сегодняшней пары! 🎪

⛳ 💻  Я предлагаю вам реализовать свою версию  [векторных романов Б. Орехова](https://habr.com/ru/post/326380/). В базовом варианте предлагаю делать упрощенно: без восстановления морфологической формы после замены слова на его векторный синоним (можем для корректности назвать это семантическим ассоциатом или квази-синонимом). Ну то есть заменяем "бегемотом" на "гиппопотам" (или что там выдаст word2vec), но форму "гиппопотамом" уже не восстанавливаем. 

А кто чувствует в себе силы — делайте полную версию, с восстановлением исходной грамматической формы "гиппопотамом". За это оценка будет выше. Ну и результат у вас будет гораздо прикольнее, потому что текст будет довольно читабелен.

К <b><s>10</s> 16 мая</b> нужно сдать любое крупное произведение русской литературы, в котором слова заменены на их векторные семантические ассоциаты. Ну и, конечно, код, который это делает. Код из этой тетрадки можно переиспользовать.

## 4. Тренируем свою модель в gensim (остается на домашние эксперименты и следующий раз)

Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий. Для этого используется стандартная питоновская библиотека **logging**.

In [ ]:
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

На вход модели мы даем наш обработанный текстовый файл (либо любой другой текст, важно лишь, что каждое предложение должно быть на отдельной строчке).

In [ ]:
f = 'harms_processed.txt'
data = gensim.models.word2vec.LineSentence(f)


Инициализируем модель. Параметры в скобочках:
* data - данные, 
* size - размер вектора, 
* window - размер окна наблюдения,
* min_count - мин. частотность слова в корпусе, которое мы берем,
* sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

In [ ]:
model = gensim.models.Word2Vec( data, size=500, window=10, min_count=2, sg=0, iter=10)

In [ ]:
# вариант для новой версии gensim:
# model = gensim.models.Word2Vec(data, vector_size=500, window=10, min_count=2, sg=0)

Мы создаем модель, в которой размерность векторов — 500, размер окна наблюдения — 10 слов, алгоритм обучения — CBOW, слова, встретившиеся в корпусе только 1 раз, не используются. После тренировки модели можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировать.

In [ ]:
model.init_sims(replace=True)

Смотрим, сколько в модели слов:

In [ ]:
print(len(model.wv.vocab))

И сохраняем!

In [ ]:
model.save('my.model')

In [ ]:
model.most_similar ('королева_NOUN')

In [ ]:
# для новой версии генсим, где не работает просто most_similar:
model.wv.most_similar ('карандаш_NOUN')

### ⛳ 💻 Задание

Задание: попробуйте обучить свою модель на каких-нибудь текстах. Например, [вот текст "Войны и мира"](https://github.com/dhhse/dh2020/blob/master/data/wap_w2v.txt), в котором каждое предложение с новой строки. Обучите модель на нем. Исследуйте близости слов. 

*   Подсказка: вам точно понадобится `gensim.models.word2vec.LineSentence` (см. выше) чтобы преобразовать текст в формат, который можно передавать для обучения модели
*   Подсказка 2: а еще вам понадобится `gensim.models.Word2Vec` (см. выше)
*   Подсказка 3: без остального в принципе можно обойтись. Но если подать тексты сырыми, то модель обучится на токенах. "Кот", "кот" и "кота" будут для нее разными словами. Зато "печь" будет одним словом вне завимисимости от части речи. Поэтому можно еще воспользоваться функцией `tag_ud` выше и обучить модель как у русвекторес. Но осторожно. UDPipe-ом обработать всю Войну и Мир — это минут 6-7.

Без предобработки ваша модель сможет как-то вот так: 

<img src = "https://github.com/dhhse/dh2020/raw/master/pics/napoleon_token.png">
<img src = "https://github.com/dhhse/dh2020/raw/master/pics/war_token.png">

C предобработкой будет как в моделях русвекторес: лемма с пос-тегом. И это на русском работает осмысленнее обычно: 

<img src = "https://github.com/dhhse/dh2020/raw/master/pics/war_lemma_pos.png">

In [ ]:
!wget 'https://github.com/dhhse/dh2020/raw/master/data/wap_w2v.txt' 

--2021-05-13 15:53:10--  https://github.com/dhhse/dh2020/raw/master/data/wap_w2v.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhhse/dh2020/master/data/wap_w2v.txt [following]
--2021-05-13 15:53:10--  https://raw.githubusercontent.com/dhhse/dh2020/master/data/wap_w2v.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5171830 (4.9M) [text/plain]
Saving to: ‘wap_w2v.txt’

wap_w2v.txt         100%[===================>]   4.93M  25.6MB/s    in 0.2s    

2021-05-13 15:53:11 (25.6 MB/s) - ‘wap_w2v.txt’ saved [5171830/5171830]



In [ ]:
wap = 'wap_w2v.txt'
data = gensim.models.word2vec.LineSentence(wap)

In [ ]:
model_wap = gensim.models.Word2Vec(data, size=500, window=10, min_count=2, sg=0)

2021-05-13 15:54:45,536 : INFO : collecting all words and their counts
2021-05-13 15:54:45,539 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-05-13 15:54:45,628 : INFO : PROGRESS: at sentence #10000, processed 161603 words, keeping 39106 word types
2021-05-13 15:54:45,717 : INFO : PROGRESS: at sentence #20000, processed 335781 words, keeping 65326 word types
2021-05-13 15:54:45,786 : INFO : collected 81592 word types from a corpus of 462159 raw words and 27123 sentences
2021-05-13 15:54:45,789 : INFO : Loading a fresh vocabulary
2021-05-13 15:54:45,864 : INFO : effective_min_count=2 retains 30236 unique words (37% of original 81592, drops 51356)
2021-05-13 15:54:45,866 : INFO : effective_min_count=2 leaves 410803 word corpus (88% of original 462159, drops 51356)
2021-05-13 15:54:45,959 : INFO : deleting the raw counts dictionary of 81592 items
2021-05-13 15:54:45,964 : INFO : sample=0.001 downsamples 36 most-common words
2021-05-13 15:54:45,966 : INFO :

In [ ]:
model_wap.most_similar('Наташа')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('долго', 0.9984106421470642),
 ('Соню,', 0.9982855319976807),
 ('что-то', 0.9982360601425171),
 ('нее,', 0.9981175065040588),
 ('за', 0.9978007078170776),
 ('Наташе,', 0.9977572560310364),
 ('грудь', 0.997657299041748),
 ('заметила,', 0.9976304769515991),
 ('слушал', 0.9974093437194824),
 ('где', 0.9973325729370117)]

In [ ]:
len(model_wap.wv.vocab)

30236

In [ ]:
text = open('wap_w2v.txt', 'r', encoding='utf-8').read()
processed_text = tag_ud(text=text, modelfile=modelfile)
print(processed_text[:350])
with open('wap_w2v_processed.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)


Loading the model...
Processing input...


то_PRON первый_ADJ

часть_NOUN первый_ADJ

i_NUM

Eh_PROPN bien_X mon_X prince_X
Gênes_PROPN et_X Lucques_PROPN ne_X sont_X plus_X que_X des_X apanages_X des_X поместье_NOUN de_X la_X famille_X Buonaparte_PROPN
Non_PROPN je_X vous_X préviens_X que_X si_X vous_X ne_X me_X dites_X pas_X que_X nous_X avons_X la_X guerre_X si_X vous_X vous_X permettez_


In [ ]:
data = gensim.models.word2vec.LineSentence('wap_w2v_processed.txt')

In [ ]:
model_wap_lemmas_pos = gensim.models.Word2Vec(data, size=300, window=10, min_count=2, sg=0)

2021-05-13 16:07:24,984 : INFO : collecting all words and their counts
2021-05-13 16:07:24,987 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-05-13 16:07:25,076 : INFO : PROGRESS: at sentence #10000, processed 156252 words, keeping 15798 word types
2021-05-13 16:07:25,181 : INFO : PROGRESS: at sentence #20000, processed 324927 words, keeping 24155 word types
2021-05-13 16:07:25,263 : INFO : collected 29037 word types from a corpus of 447977 raw words and 27064 sentences
2021-05-13 16:07:25,269 : INFO : Loading a fresh vocabulary
2021-05-13 16:07:25,300 : INFO : effective_min_count=2 retains 13506 unique words (46% of original 29037, drops 15531)
2021-05-13 16:07:25,301 : INFO : effective_min_count=2 leaves 432446 word corpus (96% of original 447977, drops 15531)
2021-05-13 16:07:25,343 : INFO : deleting the raw counts dictionary of 29037 items
2021-05-13 16:07:25,346 : INFO : sample=0.001 downsamples 50 most-common words
2021-05-13 16:07:25,348 : INFO :

In [ ]:
model_wap_lemmas_pos.most_similar ('Наташа_PROPN')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Пьер_PROPN', 0.9753262996673584),
 ('Марья_PROPN', 0.9669135808944702),
 ('осведомляться_VERB', 0.9598015546798706),
 ('удивление_NOUN', 0.9585548639297485),
 ('письмо_NOUN', 0.9561311602592468),
 ('отъезд_NOUN', 0.9557530283927917),
 ('знакомый_NOUN', 0.9538062810897827),
 ('холодность_NOUN', 0.9526064395904541),
 ('разговор_NOUN', 0.9524383544921875),
 ('смутный_ADJ', 0.9505127668380737)]

In [ ]:
len(model_wap_lemmas_pos.wv.vocab)

13506


## FastText: эмбеддинги n-граммов

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грамов. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

### Для работы с fasttext-моделью придется обновить gensim

Корректная работа с fasttext-моделями гарантируется от версии 3.7.2, а в колабе по умолчанию генсим 3.6.0

In [ ]:
import  gensim
gensim.__version__

In [ ]:
!pip install --upgrade gensim

После этого надо перезапустить среду (колаб и сам вам предложит это сделать)

In [ ]:
import gensim

In [ ]:
gensim.__version__

### Отлично, теперь возьмем русскую фасттекст-модель 
из уже известной нам [коллекции](https://rusvectores.org/ru/models/) RusVectores

In [ ]:
!wget 'http://vectors.nlpl.eu/repository/20/214.zip'

В генсим её надо загружать чуть иначе, чем word2vec-овскую модель. Надо сначала распаковать архив:

In [ ]:
!unzip '214.zip'

In [ ]:
fasttext_model = gensim.models.KeyedVectors.load('model.model')

### косинусная близость на примерах

In [ ]:
fasttext_model.most_similar ('кравать') # попробовать разные опечатки

In [ ]:
fasttext_model.most_similar ('котэ')

In [ ]:
fasttext_model.most_similar ('некузявый')

In [ ]:
fasttext_model.most_similar ('лебедиво')

In [ ]:
fasttext_model.most_similar ('lol') # латиницу русская модель не делает, делает веселое

## Как выйти за пределы слов и применить это на практике?

Как я уже говорил, векторные модели сделали не для того, чтобы веселиться с векторной математикой (ну или не только для этого). В первую очередь это супер-полезный способ **векторизации текста** для практических задач комплингвистики:


*   Классификация текстов
*   Извлечение информации (которое часто сводится к задаче классификации слов или их последовательностей)
*   Анализ тональности
*   И прочее

Если объектом является не отдельное слово, а предложение или текст (так бывает часто, например, при решении все той же задачи классификации текстов) то общая идея такая: весь текст превращается в вектор (набор чиселок), которые как-то зависят от векторов его слов (множества наборов чиселок). 

Самый очевидный вариант — сложить вектора всех слов текста или взять средний вектор всех слов текста.



In [ ]:
import numpy as np

In [ ]:
fasttext_model.vectors.shape

(347295, 300)

In [ ]:
np.average (model.wv['глокая'], model.wv['куздра'], model.wv['штеко'] )

Чуть более тонкое есть в алгоритме doc2vec — его сделали те же люди, что и word2vec. Они придумали как бы добавлять еще одно псевдо-слово в контекст при обучении векторов слов для word2vec. В результате у нас после обучения кроме векторов слов есть еще один вектор той же размерности, который как бы побывал в контексте всех слов данного документа. Он и выдается в качестве вектора (эмбеддинга) документа. 

<img src = "https://miro.medium.com/max/535/0*x-gtU4UlO8FAsRvL.">

Реализация `doc2vec`



# Заключение

В этом тьюториале мы постарались разобраться с тем, как работать с семантическими векторными моделями и библиотекой **gensim**. Теперь вы можете:
* использовать готовые модели векторной семантики,осуществлять простые операции над векторами слов.
* осуществлять предобработку текстовых данных, что может пригодиться во многих задачах обработки естественного языка;
* тренировать векторные семантические модели. Формат моделей совместим с моделями, представленными на веб-сервисе **RusVectōrēs**;



## Что мы не затронули?



*   Оценка качества моделей (как тут считать точность-полноту-F-меру). См например в [этой тетрадке](https://github.com/ancatmara/data-science-nlp/blob/master/2.%20Embeddings.ipynb) у Оксаны оценку качества на задачах оценки семантической близости и поиска аналогии (Москва Россия Берлин Германия).
*   Применение для реальных задач классификации текстов. См. в [этой тетрадке](https://github.com/mannefedov/compling_nlp_hse_course/blob/master/2020/Embeddings.ipynb) Миши Нефедова.

